In [19]:
#Importing necessary libraries
import requests
import re
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import datetime as dt
import time
import csv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from time import sleep
from random import randint

browser= webdriver.Chrome()

#Safecity URL
url="https://maps.safecity.in/reports"

browser.get(url)

headers = {"Accept-Language": "en-US, en;q=0.5"}

results = requests.get(url, headers=headers)

link_tags = []
title = []
short_desc = []
long_desc = []
date = []
location = []
tags = []

delay = 15 # seconds

i = 1

#Total number of pages
while i < 645:
    
    soup = BeautifulSoup(browser.page_source,"html.parser")

    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'r_description')))

    safecity_div = soup.find_all('div', class_='rb_report unverified')

#Getting incident IDs
    for container in safecity_div:
        link_tags.append(container.get('id'))
        
#Getting incident titles
    for container in safecity_div:
        if container.h3.a is not None:
            header = container.h3.a.text
            header = re.sub('\s+',' ',header)
            title.append(header)
        else:
            title.append(0)

#Getting user reported descriptions
    for container in safecity_div:
        if container.find('div',class_='r_description') is not None:
            short_context = container.find('div',class_='r_description').text
            short_desc.append(short_context)
        else:
            short_desc.append(0)

#Getting incident dates
    for container in safecity_div:
        if container.find('p',class_='r_date r-3 bottom-cap') is not None:
            dtime = container.find('p',class_='r_date r-3 bottom-cap').text
            dtime = dt.datetime.strptime(dtime, "%H:%M %b %d, %Y")
            date.append(dtime)      
        else:
            date.append(0)

#Getting incident locations
    for container in safecity_div:
        if container.find('p',class_='r_location') is not None:
            loc = container.find('p',class_='r_location').text
            location.append(loc)
        else:
            location.append(0)

#Getting incident tags
    for container in safecity_div:
        if container.find('span',class_='r_cat-desc') is not None:
            tag = container.find('span',class_='r_cat-desc').text
            tags.append(tag)
        else:
            tags.append(0)

#Navigating to next page
    element = WebDriverWait(browser, 50).until(EC.element_to_be_clickable((By.CLASS_NAME, 'next')))
    element.click()
    time.sleep(5)
    i+=1

In [11]:
#Using selenium to get long user reported descriptions from each link within page
for link in link_tags:
        first_link = browser.find_element_by_xpath("//*[@id=\"{}\"]/div[2]/h3/a[1]".format(str(link)))
        main_window = browser.current_window_handle
        first_link.send_keys(Keys.CONTROL + Keys.RETURN)
        first_result = ui.WebDriverWait(browser, 15).until(lambda browser: browser.find_element_by_class_name('report-description-text'))
        popup_content=browser.find_element_by_class_name('report-description-text').text
        if popup_content is not None:
            content.append(popup_content)
            browser.back()
        else:
            content.append(0)
            browser.back()
        browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)
        browser.switch_to_window(main_window)
        sleep(2)
        browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'w')
        browser.switch_to_window(main_window)

20

In [20]:
len(location)

12681

In [18]:
container.find('div',class_='r_cat-de sc') is not None

False

In [21]:
sh_data = pd.DataFrame(columns=['incident_id','title','short_desc','date','location','tags'])
sh_data

,incident_id,title,short_desc,date,location,tags


In [22]:
sh_data['incident_id'] = link_tags
sh_data['title'] = title
sh_data['short_desc'] = short_desc
sh_data['date'] = date
sh_data['location'] = location
sh_data['tags'] = tags

In [23]:
sh_data

,incident_id,title,short_desc,date,location,tags
0,incident_13635,Report on facebook,I'm a 58 year old woman and not used to seein...,2020-10-22 18:28:00,"Dhaula Kuan, Dhaula Kuan Enclave I, Dhaula Kua...",Indecent exposure
1,incident_12795,Community Kitchen,"Ishtam Catering, Free or subsidised food by d...",2020-09-04 15:45:00,"Vettom, Kerala",COVID19 - Food
2,incident_12797,Community Kitchen,"CDS, Free or subsidised food by district admi...",2020-09-04 15:45:00,"Vettathur, Kerala",COVID19 - Food
3,incident_12796,Community Kitchen,"Kaithangu Catering and canteen, Free or subsi...",2020-09-04 15:45:00,"Vettikkavala, Kerala",COVID19 - Food
4,incident_12798,Community Kitchen,"Jbs School, Venmony, Chengannur, Free or subs...",2020-09-04 15:45:00,"Venmony, Kerala",COVID19 - Food
5,incident_13625,cyber bully,There is a facebook group of people who bully...,2020-06-21 11:45:00,"delhi green park 16 delhi green park 16, Pocke...",Online Harassment
6,incident_13618,Sexual harassment,a boy with whom I was in relationship with ca...,2020-06-16 15:00:00,"Dwarka, Dwarka, Gujarat 361335, India",Touching /Groping
7,incident_13620,Domestic Violence,My husband and my in laws trouble me a lot fo...,2020-06-10 20:00:00,"Unnamed Road, Civil Line, Tikamgarh, Madhya Pr...",Domestic Violence
8,incident_13619,Domestic Violence,My married pregnant cousin died in suspicious...,2020-06-10 12:19:00,"ADB PWD Colony, Sector 16A, Faridabad, Haryana...",Domestic Violence
9,incident_13560,Community Kitchen,"G M L P School, Koduvally\tFree or subsidised...",2020-05-24 23:09:00,"Koduvally-Omassery Rd, Koduvally, Kerala 67357...",COVID19 - Food


In [25]:
sh_data.to_csv('safecity.csv',encoding='utf-8')